# Analysis of the imaging experiment
mouse name: 067 (or 057?)  
task: NearFarLong  
experimentalist: Snezana Raus-Balind  

### 1. Load the necessary object class
We use a custom-made class, ImagingSessionData, that will contain all behavioral and imaging data

In [1]:
from ImageAnal import *
%matplotlib widget

### 2. Tell python where the data is
The required file structure is the following:  
- All data should be in the data folder
- Within the data folder separate subfolders are needed for each mouse. Folder name starts with the **name** of the mouse.
- For each mouse there should be at least two folders: one for the **imaging** data and one for the **behavioral** data.
- The behavioral folder is named as `MouseName_TaskName` - so we need a separate folder for each different task
- The behavioral log files are in separate subfolders named by the experiment's start time within the behavioral folder - e.g. `2021-02-03_10-15-50`
- The imaging folder is named as `MouseName_imaging`
- The suite2p imaging files are also in separate folders for each experiment below the imaging folder. 

In [2]:
datapath = os.getcwd() + '/' #current working directory - look for data and strings here!
date_time = '2021-02-05_11-32-15' # date and time of the imaging session
name = 'srb067' # mouse name
task = 'NearFarLong' # task name

## locate the suite2p folder
suite2p_folder = datapath + 'data/' + name + '_imaging/Suite2P_1_06-07-2021_manual_ROI/'

## the name and location of the imaging log file
imaging_logfile_name = suite2p_folder + 'srb057_TSeries-02052021-0813-002.xml'

## the name and location of the trigger voltage file
TRIGGER_VOLTAGE_FILENAME = suite2p_folder + 'srb057_TSeries-02052021-0813-002_Cycle00001_VoltageRecording_001.csv'

### 3. Load all the data - this takes ~20 secs in my computer
Python looks for the data in the specified folders. It loads the behavioral data (position, lick and rewards) as well as the imaging data. It calculates the activity of the cells as a function of position in the different corridors and calculates their spatial tuning measures and corridor selectivity.  
The name of the object that contains all the data is `D1` here - Data 1.

In [3]:
# 3. load all the data - this taks ~20 secs in my computer
D1 = ImagingSessionData(datapath, date_time, name, task, suite2p_folder, imaging_logfile_name, TRIGGER_VOLTAGE_FILENAME)#, startendlap=[27, 99])


trigger logfile loaded
trigger voltage signal loaded
triggers after: 40
n_extra_indexes 5
candidate log indexes [0, 99, 198, 229]
min recorded trigger length: 0.011800000000008026
relevant behavior located, lap time of the first frame: 984.608327 , log reference index: 99
slight warning - testing some late candidates failed
suite2p data loaded
corrected offset: 984.6053269999994 voltage_delay: 0.00300000000061118
suite2p time axis loaded
calculating dF/F and SNR...
SNR done
dF/F calculated for cell ROI-s
ExpStateMachineLog time interval > 1s:  40  times
############################################################
substage change detected!
first lap in substage  ['1'] is lap 3 , which started at t 417.73697223613453
the time of the change in imaging time is:  -566.8713547638655
############################################################
calculating rate, reliability and Fano factor...
calculating Skaggs spatial info...
calculating proportion of active laps...
calculating proportion of 

The behavior is divided into laps (trials or runs). You can check the **number of laps** and which lap is associated with imaging data in the following way:

In [4]:
print(D1.n_laps)
print(D1.i_Laps_ImData)

229
[ 83  84  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99 100
 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118
 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136
 137 138]


So we have 229 laps and laps 83-138 contain imaging data.  
### 4. Plotting the behavioral data
You can plot the behavioral data of the session:

In [5]:
#D1.plot_session()
D1.plot_session(selected_laps=D1.i_Laps_ImData)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### 5. Plot the ratemaps
First, we plot the ratemaps of some neurons. There are several options - selecting the cells, sorting and normalising the ratemaps.
- selection: here we select all active cells (activity in at least 20% of laps), but any other selection criteria 
- sorting: place fields can be sorted by either corridors
- place cells can be normalised - so the peak has the same height

In [6]:
D1.ratemaps[1].shape

(75, 4)

In [7]:
cellids = np.arange(4)
D1.plot_ratemaps(cellids = cellids, sorted=False, normalized=False)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### 6. Plot the spatial properies of the neurons

In [8]:
D1.plot_properties(cellids=cellids, interactive=False)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### 7. Calculate significance of tuning by shuffling the imaging data 

In [9]:
D1.calc_shuffle(cellids, 1000, 'shift')

loading shuffling P-values from file...
tuned cells: [array([], dtype=int64), array([], dtype=int64)]
selective cells: []
similar cells: []


In [10]:
D1.shuffle_Pvalues

array([[0.   , 0.366, 0.318, 0.367, 0.304, 0.464, 0.291, 0.148, 0.107,
        0.   , 0.   ],
       [1.   , 0.54 , 0.479, 0.32 , 0.222, 0.624, 0.309, 0.151, 0.09 ,
        0.   , 0.   ],
       [2.   , 0.489, 0.569, 0.246, 0.438, 0.967, 0.024, 0.409, 0.479,
        0.   , 0.   ],
       [3.   , 0.017, 0.998, 0.259, 0.283, 0.644, 0.741, 0.141, 0.544,
        0.   , 0.   ]])

### 6.  Plot the activity lap by lap
We can also plot the lap by lap activity of a selected cell. Again, there are several options, but the simplest is to plot the rate as a function of position.

In [11]:
D1.plot_cell_laps(cellid=3, signal='rate') ## look at lap 20

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
D1.plot_cell_laps(cellid=3, signal='dF') ## look at lap 20

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …